In [3]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os 
from PIL import Image 
import numpy as np 

In [13]:
window=tk.Tk()
window.title("Facial Recognition System")

l1=tk.Label(window,text="Name",font=("Algerian",20))
l1.grid(column=0, row=0)
t1=tk.Entry(window,width=50,bd=5)
t1.grid(column=1, row=0)

l2=tk.Label(window,text="Age",font=("Algerian",20))
l2.grid(column=0, row=1)
t2=tk.Entry(window,width=50,bd=5)
t2.grid(column=1, row=1)

l3=tk.Label(window,text="Address",font=("Algerian",20))
l3.grid(column=0, row=2)
t3=tk.Entry(window,width=50,bd=5)
t3.grid(column=1, row=2)

b1=tk.Button(window,text="Trainig",font=("Algerian",20),bg='orange',fg='red')
b1.grid(column=0, row=4)

b2=tk.Button(window,text="Detect Face",font=("Algerian",20),bg='green',fg='white')
b2.grid(column=1, row=4)

b3=tk.Button(window,text="Generate Dataset",font=("Algerian",20),bg='pink',fg='black')
b3.grid(column=2, row=4)

window.geometry("800x200")
window.mainloop()